# side model

In [1]:
import numpy as np
import pandas as pd

from model.config import SIDE_ALL

df_features = pd.read_parquet("data/features_5h.parquet")
df_features = df_features[SIDE_ALL]
label_side = np.load("data/label_side.npy")
print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(9058, 2500)
(9058, 7)


15h_dft_dom_cycle_lag23    214
15h_dft_dom_cycle_lag22    211
15h_dft_dom_cycle_lag21    208
15h_dft_dom_cycle_lag6     163
15h_natr_dt_lag36          154
                          ... 
5h_comb_spectrum_pwr_24      0
5h_comb_spectrum_pwr_23      0
5h_comb_spectrum_pwr_22      0
5h_ac_31                     0
5h_comb_spectrum_pwr_32      0
Length: 2500, dtype: int64

In [2]:
df_features = df_features[214:]
label_side = label_side[214:]

print(df_features.shape)
print(label_side.shape)

df_features.isna().sum().sort_values(ascending=False)

(8844, 2500)
(8844, 7)


100m_ac_34                             0
15h_adaptive_stochastic_lag10          0
15h_adaptive_stochastic_dt_lag13       0
15h_adaptive_stochastic_dt_lag14       0
15h_adaptive_stochastic_dt_lag15       0
                                      ..
5h_evenbetter_sinewave_long_dt_lag2    0
5h_evenbetter_sinewave_long_dt_lag3    0
5h_evenbetter_sinewave_long_dt_lag4    0
5h_evenbetter_sinewave_long_dt_lag5    0
15h_williams_r_lag9                    0
Length: 2500, dtype: int64

In [3]:
side_features = df_features
print(side_features.shape)
side_label = (label_side[:, 6].astype(int) == 1).astype(int)
print(np.unique(side_label, return_counts=True))

train_test_split_point = int(side_features.shape[0] * 0.8)
side_features_train = side_features[:train_test_split_point]
side_features_test = side_features[train_test_split_point:]
side_label_train = side_label[:train_test_split_point]
side_label_test = side_label[train_test_split_point:]

print(np.unique(side_label_test, return_counts=True))

(8844, 2500)
(array([0, 1]), array([3562, 5282]))
(array([0, 1]), array([ 739, 1030]))


In [4]:
import lightgbm as lgb
import optuna
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score


def objective(trial):
    params = {
        "objective": "binary",
        "metric": "auc",
        "num_threads": -1,
        "verbose": -1,
        "is_unbalance": trial.suggest_categorical("is_unbalance", [True, False]),
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        # "early_stopping_round": 100,
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 300),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 500),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-4, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-4, 100),
    }
    dtrain = lgb.Dataset(side_features_train, side_label_train)
    dtest = lgb.Dataset(side_features_test, side_label_test)
    model = lgb.train(
        params,
        dtrain,
        # valid_sets=[dtest],
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(side_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(side_label_test, pred_label)
    recall = recall_score(side_label_test, pred_label)
    f1 = f1_score(side_label_test, pred_label)
    auc = roc_auc_score(side_label_test, pred_proba)
    print(f"precision: {precision}, recall: {recall}, f1: {f1}")
    return auc


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-27 14:16:22,579] A new study created in memory with name: no-name-4324bff2-d829-4ad3-a77c-daa618ca299e
[I 2025-03-27 14:16:24,162] Trial 0 finished with value: 0.8794027615381583 and parameters: {'is_unbalance': True, 'extra_trees': True, 'boosting': 'dart', 'num_leaves': 260, 'max_depth': 780, 'min_gain_to_split': 0.5564265418406479, 'min_data_in_leaf': 327, 'lambda_l1': 95.83109586070046, 'lambda_l2': 71.31682327084577, 'num_boost_round': 174}. Best is trial 0 with value: 0.8794027615381583.


precision: 0.8443983402489627, recall: 0.7902912621359224, f1: 0.8164493480441324


[I 2025-03-27 14:16:28,547] Trial 1 finished with value: 0.8897171459726474 and parameters: {'is_unbalance': True, 'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 43, 'max_depth': 401, 'min_gain_to_split': 0.6447921778716211, 'min_data_in_leaf': 332, 'lambda_l1': 97.41619989719423, 'lambda_l2': 31.09473310984301, 'num_boost_round': 1302}. Best is trial 1 with value: 0.8897171459726474.


precision: 0.8531540847983454, recall: 0.8009708737864077, f1: 0.8262393590385578


[I 2025-03-27 14:16:31,086] Trial 2 finished with value: 0.8828894990606568 and parameters: {'is_unbalance': False, 'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 197, 'max_depth': 654, 'min_gain_to_split': 0.29090135080177554, 'min_data_in_leaf': 356, 'lambda_l1': 96.60503011761872, 'lambda_l2': 96.49263734199718, 'num_boost_round': 662}. Best is trial 1 with value: 0.8897171459726474.


...

precision: 0.8345864661654135, recall: 0.8621359223300971, f1: 0.8481375358166189


[I 2025-03-27 14:26:55,028] Trial 98 finished with value: 0.9092686259311324 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 211, 'max_depth': 635, 'min_gain_to_split': 0.19189022177603435, 'min_data_in_leaf': 301, 'lambda_l1': 12.127063652605372, 'lambda_l2': 21.70858779051802, 'num_boost_round': 450}. Best is trial 85 with value: 0.9134674251481273.


precision: 0.8448753462603878, recall: 0.8883495145631068, f1: 0.8660672030288689


[I 2025-03-27 14:27:03,958] Trial 99 finished with value: 0.8822142228411525 and parameters: {'is_unbalance': False, 'extra_trees': False, 'boosting': 'dart', 'num_leaves': 278, 'max_depth': 768, 'min_gain_to_split': 0.12530278214438545, 'min_data_in_leaf': 22, 'lambda_l1': 18.00430125008154, 'lambda_l2': 60.305105924896665, 'num_boost_round': 338}. Best is trial 85 with value: 0.9134674251481273.


precision: 0.8184383819379115, recall: 0.8446601941747572, f1: 0.8313425704730053


In [5]:
study.best_params

{'is_unbalance': False,
 'extra_trees': False,
 'boosting': 'dart',
 'num_leaves': 281,
 'max_depth': 878,
 'min_gain_to_split': 0.18437770920018662,
 'min_data_in_leaf': 206,
 'lambda_l1': 15.083265465050545,
 'lambda_l2': 42.857520281163,
 'num_boost_round': 556}

In [6]:
import lightgbm as lgb

params = {
    # "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}


dtrain = lgb.Dataset(side_features_train, side_label_train)
model = lgb.train(params, dtrain)

prod_model = lgb.train(params, lgb.Dataset(side_features, side_label))

In [7]:
import plotly.express as px

res = model.predict(side_features_train)

fig = px.histogram(res, nbins=100)
fig.show()

In [9]:
import plotly.express as px
from sklearn.metrics import roc_auc_score

res = model.predict(side_features_test)
print(roc_auc_score(side_label_test, res))

fig = px.histogram(res, nbins=100)
fig.show()

0.9134674251481273


In [10]:
model.save_model("model/model_side.txt")
prod_model.save_model("model/model_side_prod.txt")

# meta model

In [9]:
import lightgbm as lgb
import numpy as np
import pandas as pd

from model.config import META_ALL, SIDE_ALL

df_features = pd.read_parquet("data/features_5h.parquet")
meta_label = np.load("data/label_meta.npy")

model_side = lgb.Booster(model_file="model/model_side.txt")
side_model_res = model_side.predict(df_features[SIDE_ALL])
df_features["side_model_res"] = side_model_res

df_features = df_features[META_ALL]
df_features.isna().sum(axis=0).sort_values(ascending=False)

15h_dft_dom_cycle_dt_lag36     256
15h_dft_dom_cycle_dt_lag35     253
15h_dft_dom_cycle_ddt_lag34    253
15h_dft_dom_cycle_lag28        229
15h_dft_dom_cycle_lag26        223
                              ... 
5h_comb_spectrum_pwr_33          0
5h_comb_spectrum_pwr_5           0
5h_comb_spectrum_pwr_6           0
5h_comb_spectrum_pwr_7           0
side_model_res                   0
Length: 2500, dtype: int64

In [10]:
df_features = df_features[256:]
meta_label = meta_label[256:]

print(df_features.shape)

assert df_features.shape[0] == meta_label.shape[0]
print(meta_label.shape)

df_features.isna().sum(axis=0).sort_values(ascending=False)

(8802, 2500)
(8802, 7)


100m_ac_12                           0
15h_adaptive_stochastic_ddt_lag36    0
15h_adaptive_stochastic_ddt_lag23    0
15h_adaptive_stochastic_ddt_lag24    0
15h_adaptive_stochastic_ddt_lag25    0
                                    ..
5h_dft_dom_cycle_lag24               0
5h_dft_dom_cycle_lag25               0
5h_dft_dom_cycle_lag26               0
5h_dft_dom_cycle_lag27               0
side_model_res                       0
Length: 2500, dtype: int64

In [11]:
meta_features = df_features
print(meta_features.shape)
meta_label = meta_label[:, 6].astype(int)

train_test_split_point = int(meta_features.shape[0] * 0.8)
meta_features_train = meta_features[:train_test_split_point]
meta_features_test = meta_features[train_test_split_point:]
meta_label_train = meta_label[:train_test_split_point]
meta_label_test = meta_label[train_test_split_point:]

print(np.unique(meta_label_train, return_counts=True))
print(np.unique(meta_label_test, return_counts=True))

(8802, 2500)
(array([0, 1]), array([  58, 6983]))
(array([0, 1]), array([ 275, 1486]))


In [12]:
import optuna
from sklearn.metrics import (
    f1_score,
    fbeta_score,
    precision_score,
    recall_score,
    roc_auc_score,
)


def objective(trial):
    params = {
        "objective": "binary",
        "is_unbalance": True,
        "num_threads": -1,
        "verbose": -1,
        "extra_trees": trial.suggest_categorical("extra_trees", [True, False]),
        "boosting": trial.suggest_categorical("boosting", ["gbdt", "dart"]),
        "num_leaves": trial.suggest_int("num_leaves", 31, 500),
        "max_depth": trial.suggest_int("max_depth", 30, 1000),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 1e-8, 1),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 20, 300),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 100),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 100),
    }
    dtrain = lgb.Dataset(meta_features_train, meta_label_train)
    # dtest = lgb.Dataset(meta_features_test, meta_label_test)
    model = lgb.train(
        params,
        dtrain,
        num_boost_round=trial.suggest_int("num_boost_round", 100, 2000),
    )
    pred_proba = model.predict(meta_features_test)
    pred_label = (pred_proba > 0.5).astype(int)
    precision = precision_score(meta_label_test, pred_label)
    recall = recall_score(meta_label_test, pred_label)
    f1 = f1_score(meta_label_test, pred_label)
    fbeta = fbeta_score(meta_label_test, pred_label, beta=1.2, average="weighted")
    auc = roc_auc_score(meta_label_test, pred_proba, average="weighted")
    print(
        f"precision: {precision}, recall: {recall}, f1: {f1}, auc: {auc}, fbeta: {fbeta}"
    )
    return fbeta


study = optuna.create_study(
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(),
    sampler=optuna.samplers.TPESampler(),
)
study.optimize(objective, n_trials=100, n_jobs=1)

/opt/homebrew/Caskroom/miniforge/base/envs/jesse/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-27 14:40:22,232] A new study created in memory with name: no-name-85d9e3e8-84e0-4f6b-9a12-1c7429ff4c1f
[I 2025-03-27 14:40:24,656] Trial 0 finished with value: 0.808905676989424 and parameters: {'extra_trees': False, 'boosting': 'gbdt', 'num_leaves': 471, 'max_depth': 556, 'min_gain_to_split': 0.6636096707096457, 'min_data_in_leaf': 68, 'lambda_l1': 11.641739840517435, 'lambda_l2': 63.39765695713846, 'num_boost_round': 335}. Best is trial 0 with value: 0.808905676989424.


precision: 0.8812664907651715, recall: 0.8990578734858681, f1: 0.8900732844770153, auc: 0.7604453688975896, fbeta: 0.808905676989424


[I 2025-03-27 14:40:29,084] Trial 1 finished with value: 0.8165754662089106 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 194, 'max_depth': 836, 'min_gain_to_split': 0.43932450459194866, 'min_data_in_leaf': 95, 'lambda_l1': 67.554376494476, 'lambda_l2': 18.57219837677175, 'num_boost_round': 1389}. Best is trial 1 with value: 0.8165754662089106.


precision: 0.8752374920835972, recall: 0.9300134589502019, f1: 0.901794453507341, auc: 0.7638786247399976, fbeta: 0.8165754662089106


[I 2025-03-27 14:40:33,044] Trial 2 finished with value: 0.8092981515382168 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 397, 'max_depth': 844, 'min_gain_to_split': 0.7948417942948286, 'min_data_in_leaf': 172, 'lambda_l1': 61.84588310074576, 'lambda_l2': 12.287323333640472, 'num_boost_round': 1213}. Best is trial 1 with value: 0.8165754662089106.


...

precision: 0.9076576576576577, recall: 0.8135935397039031, f1: 0.85805535841022, auc: 0.7376752722378564, fbeta: 0.7871971254847862


[I 2025-03-27 14:49:15,615] Trial 98 finished with value: 0.8071189623838444 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 218, 'max_depth': 808, 'min_gain_to_split': 0.17711362200174363, 'min_data_in_leaf': 66, 'lambda_l1': 58.77302194989469, 'lambda_l2': 65.33839968176713, 'num_boost_round': 1648}. Best is trial 78 with value: 0.817577388149367.


precision: 0.8620060790273556, recall: 0.9542395693135935, f1: 0.9057809006707123, auc: 0.7483763611892817, fbeta: 0.8071189623838444


[I 2025-03-27 14:49:20,381] Trial 99 finished with value: 0.8113412406851686 and parameters: {'extra_trees': True, 'boosting': 'gbdt', 'num_leaves': 313, 'max_depth': 742, 'min_gain_to_split': 0.9043885318808558, 'min_data_in_leaf': 130, 'lambda_l1': 62.858859003787984, 'lambda_l2': 55.4515080042147, 'num_boost_round': 1530}. Best is trial 78 with value: 0.817577388149367.


precision: 0.8742820676451819, recall: 0.9219380888290714, f1: 0.8974778905994104, auc: 0.7635384803621681, fbeta: 0.8113412406851686


In [13]:
study.best_params

{'extra_trees': True,
 'boosting': 'gbdt',
 'num_leaves': 248,
 'max_depth': 765,
 'min_gain_to_split': 0.4162584712133694,
 'min_data_in_leaf': 89,
 'lambda_l1': 73.52330787362555,
 'lambda_l2': 58.446881069785526,
 'num_boost_round': 1676}

In [14]:
import lightgbm as lgb

params = {
    "is_unbalance": True,
    "objective": "binary",
    "num_threads": -1,
    "verbose": -1,
    **study.best_params,
}

model = lgb.train(
    params,
    lgb.Dataset(meta_features_train, meta_label_train),
)

prod_model = lgb.train(params, lgb.Dataset(meta_features, meta_label))

In [15]:
import plotly.express as px

res = model.predict(meta_features_test)
pred_label = (res > 0.5).astype(int)

fig = px.histogram(res, nbins=100)
fig.show()

In [16]:
model.save_model("model/model_meta.txt")
prod_model.save_model("model/model_meta_prod.txt")